# Arguments

In [5]:
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

warmup_model_path = 'model-warmup.h5'
model_save_path = 'model-final.h5'
checkpoint_path = 'checkpoints-finetuning.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# --------------------------------------------------
<<<<<<< local
num_of_epochs = 20
start_epoch = 0
=======
num_of_epochs = 30
start_epoch =  151 - 50 - 36
>>>>>>> remote
# --------------------------------------------------


# Select training classes

In [6]:
class_paths_training = ['../datasets/self/___Early_blight', '../datasets/self/___Appids', '../datasets/self/___Leaf_miner', '../datasets/self/___Curly_top_virus']

# Load dataset

In [7]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.resize_image_preprocessor import resizeImagePreprocessor
from utils.preprocessors.img_to_array_preprocessor import ImgToArrayPreprocessor
from utils.io.dataset_loader import DatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

def load_datasets(path_list):
    # Load image paths
    image_paths = []
    print("[INFO] loading images...")
    for path in path_list:
        image_paths.extend(list(paths.list_images(path)))
        
    # Get unique classnames

    class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
    class_names = [str(x) for x in np.unique(class_names)]

    # Initial image preprocessing
    aap = resizeImagePreprocessor(input_width, input_height)
    iap= ImgToArrayPreprocessor()

    #Load image data and perform image data preprocessing
    dl = DatasetLoader(preprocessors=[aap,iap])
    (data,labels)  = dl.load(image_paths,verbose=500)
    data = data.astype("float") / 255.0


    # train test split
    (train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

    # convert the labels from integers to vectors
    train_y = LabelBinarizer().fit_transform(train_y)
    test_y = LabelBinarizer().fit_transform(test_y)
    
    return (train_x,test_x,train_y,test_y, class_names)

(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_training)



[INFO] loading images...
[INFO]: Processed 500/4401
[INFO]: Processed 1000/4401
[INFO]: Processed 1500/4401
[INFO]: Processed 2000/4401
[INFO]: Processed 2500/4401
[INFO]: Processed 3000/4401
[INFO]: Processed 3500/4401
[INFO]: Processed 4000/4401


# Load model from disk

In [8]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)

OSError: Unable to open file (file signature not found)

# Print index of layers

In [ ]:
# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {:5}\t{:30}{}".format(i, layer.name, layer.__class__.__name__))

# Unfreeze final CONV layers

In [ ]:
for layer in model.layers[:7]:
   layer.trainable = False
for layer in model.layers[7:]:
   layer.trainable = True
print('unfrozen')

# Compile model

In [ ]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.0001, momentum=0.09)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


# Checkpoints

In [ ]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [ ]:
#load checkpoints if existing

import os

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)

# Training Monitor

In [ ]:
from utils.callbacks.monitor import Monitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

callbacks.append(Monitor(fig_path, json_path, start_epoch))

# Fit model

In [ ]:
H = model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/30
103/103 [==============================] - 74s 714ms/step - loss: 0.0686 - accuracy: 0.9767 - val_loss: 0.0520 - val_accuracy: 0.9918

Epoch 00001: val_loss improved from inf to 0.05204, saving model to checkpoints-finetuning.hdf5
Epoch 2/30
103/103 [==============================] - 64s 623ms/step - loss: 0.0826 - accuracy: 0.9746 - val_loss: 0.0625 - val_accuracy: 0.9882

Epoch 00002: val_loss did not improve from 0.05204
Epoch 3/30
103/103 [==============================] - 64s 626ms/step - loss: 0.0769 - accuracy: 0.9737 - val_loss: 0.0518 - val_accuracy: 0.9882

Epoch 00003: val_loss improved from 0.05204 to 0.05177, saving model to checkpoints-finetuning.hdf5
Epoch 4/30
103/103 [==============================] - 64s 622ms/step - loss: 0.0820 - accuracy: 0.9758 - val_loss: 0.0518 - val_accuracy: 0.9909

Epoch 00004: val_loss did not improve from 0.05177
Epoch 5/30
103/103 [==============================] - 64s 623ms/step - loss: 0.0701 - accuracy: 0.9816 - val_loss: 0.0

# Save model

In [ ]:
model.save(model_save_path)

# Evaluate with 20%

In [ ]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

<<<<<<< local <removed>


[INFO] evaluating after initialization...
                    precision    recall  f1-score   support

         ___Appids     0.9878    1.0000    0.9939       243
___Curly_top_virus     0.9817    0.9817    0.9817       109
   ___Early_blight     0.9948    0.9897    0.9922       388
     ___Leaf_miner     0.9972    0.9945    0.9958       361

       avg / total     0.9928    0.9927    0.9927      1101



>>>>>>> remote <modified: >


# Evaluate with validation images

In [ ]:
class_paths_validation = ['../datasets/validation/___Early_blight', '../datasets/validation/___Appids', '../datasets/validation/___Leaf_miner', '../datasets/self/___Curly_top_virus']

In [ ]:
(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_validation)

from sklearn.metrics import classification_report

print("[INFO] evaluating with validation set...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

<<<<<<< local <removed>


[INFO] loading images...
[INFO]: Processed 500/599
[INFO] evaluating with validation set...
                    precision    recall  f1-score   support

         ___Appids     1.0000    0.6000    0.7500         5
___Curly_top_virus     0.9739    1.0000    0.9868       112
   ___Early_blight     0.7059    1.0000    0.8276        12
     ___Leaf_miner     1.0000    0.7143    0.8333        21

       avg / total     0.9570    0.9467    0.9447       150



>>>>>>> remote <modified: >
